## Notes

* Error while making sklearn pipeline >> All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' '<keras_preprocessing.text.Tokenizer object at 0x00000221021B8248>' (type <class 'keras_preprocessing.text.Tokenizer'>) doesn't
##### 2
* At 2 very important how to read GLoVe with Pandas and csv module
* The burning issue of NaN in matrix fucked up the whole model. Loss became NaN

## 1

In [109]:
#Keras one hot encoding
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [44]:
#tf.keras.backend.clear_session()  # For easy reset of notebook state.
samples=['The cat sat on the mat','The dog ate my homework','The sat dog my','You should user these utilities']
tokenizer=Tokenizer(num_words=14)
tokenizer.fit_on_texts(samples)
one_hot=tokenizer.texts_to_matrix(samples,mode='binary') #1 in places of words that are present. You multiply that with the word embedding matrix

In [45]:
word_index=tokenizer.word_index
one_hot
word_index

<hr size="12">

##### Learning word embeddings to classify imdb data to be positive or negative review

In [84]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
max_word=10000
maxlen=20 #number words in each sentence to consider
(xtrain,ytrain),(xtest,ytest)=imdb.load_data(num_words=max_word)
xtrain=preprocessing.sequence.pad_sequences(xtrain,maxlen=maxlen) #cutting off sentence to 20 words or padding till 20
xtest=preprocessing.sequence.pad_sequences(xtest,maxlen=maxlen)

In [82]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

model=Sequential()
model.add(Embedding(10000,8,input_length=maxlen))
model.add(Flatten())
#model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.summary()
history=model.fit(xtrain,ytrain,epochs=10,batch_size=32,validation_split=0.2)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_8 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               20608     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 100,737
Trainable params: 100,737
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 91us/sample - loss: 0.5864 - acc: 0.6751 - val_loss: 0.5073 - val_acc: 0.7396
Epoch 2/10
20000/20000 [==============================] - 

In [83]:
results=model.evaluate(xtest,ytest)
print("Test loss : Test Accuracy ", results)

25000/25000 [==============================] - 1s 32us/sample - loss: 0.8780 - acc: 0.7076
Test loss : Test Accuracy  [0.8780133069038392, 0.7076]


<hr size="12">

## 2
##### Full model with word embeds


In [92]:
#Getting the data
import os
data=[]
labels=[]
for l_type in ['pos','neg']:
    dirname=os.path.join("",l_type)
    for file in os.listdir(dirname):
        if file[-4:]=='.txt':
            f=open(os.path.join(dirname,file),encoding="utf8")
            data.append(f.read())
            f.close()
            if l_type=='pos':
                labels.append(1)
            else:
                labels.append(0)


In [107]:
#tokenizing the data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.pipeline import make_pipeline
import numpy as np

In [142]:
maxlen=100 #reviews cut off at 
t_samples=10000
v_samples=7000
max_words=72633
tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences=tokenizer.texts_to_sequences(data) #word number
wordindex=tokenizer.word_index
print('Found %s unique tokens'%len(wordindex))

Found 72633 unique tokens


In [143]:
datapad=pad_sequences(sequences,maxlen=maxlen) #padding short sentences
labels=np.asarray(labels)
print('Shape of data tensor : ',datapad.shape)
print('Shape of label tensor : ',labels.shape)

Shape of data tensor :  (17243, 100)
Shape of label tensor :  (17243,)


In [144]:
#shuffling the data
indices=np.arange(datapad.shape[0])
np.random.shuffle(indices)
datapad=datapad[indices]
labels=labels[indices]

In [160]:
#splitting the data
xtrain=datapad[:t_samples]
ytrain=labels[:t_samples]
xval=datapad[t_samples:t_samples+v_samples]
yval=labels[t_samples:t_samples+v_samples]

import pandas as pd
import csv
glove_df = pd.read_csv("glove.6B.50d.txt", sep=" ", index_col=0, header=None,quoting=csv.QUOTE_NONE)

In [161]:
#prepping the glove
embeds={}
f=open('glove.6B.50d.txt',encoding='utf8')
for line in f:
    values=line.split(" ")
    word=values[0]
    emb=np.asarray(values[1:],dtype='float32')
    embeds[word]=emb
f.close()

In [162]:
#prepping the embedding matrix
embed_dim=50
emb_mat=np.zeros((max_words+1,embed_dim),dtype='float32')
for word, i in wordindex.items():
    emb_mat[i]=embeds.get(word)

In [169]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

model=Sequential()
model.add(Embedding(max_words+1,embed_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()
#setting weights of embedding layer
model.layers[0].set_weights([emb_mat])
model.layers[0].trainable=False

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 100, 50)           3631700   
_________________________________________________________________
flatten_12 (Flatten)         (None, 5000)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 32)                160032    
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 33        
Total params: 3,791,765
Trainable params: 3,791,765
Non-trainable params: 0
_________________________________________________________________


In [171]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
history=model.fit(xtrain,ytrain, 
                  epochs=100,
                  batch_size=64,
                  validation_data=(xval,yval))
model.save_weights('pretrained.h5')

Train on 10000 samples, validate on 7000 samples
Epoch 1/100
10000/10000 [==============================] - 1s 105us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 2/100
10000/10000 [==============================] - 1s 56us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 3/100
10000/10000 [==============================] - 1s 58us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 4/100
10000/10000 [==============================] - 1s 60us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 5/100
10000/10000 [==============================] - 1s 54us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 6/100
10000/10000 [==============================] - 1s 61us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 7/100
10000/10000 [==============================] - 1s 66us/sample - loss: nan - acc: 0.7229 - val_loss: nan - val_acc: 0.7263
Epoch 8/100


KeyboardInterrupt: 

In [165]:
history.history['acc']

[0.7229,
 0.7229,
 0.7229,
 0.7229,
 0.7229,
 0.7229,
 0.7229,
 0.7229,
 0.7229,
 0.7229]

In [173]:
z=np.sum(emb_mat)
np.isnan(z)

True